In [42]:
import requests
import html
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

class FundAPITester:
    def __init__(self):
        self.base_url = "https://api.cryptorank.io/v0"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            "Accept": "application/json"
        }
        self.timeout = 10


    def get_investments_by_country(self, slug: str, start_date: str = None, end_date: str = None):
       """Fetch and transform country investment data with robust error handling"""
       endpoint = f"{self.base_url}/fund-chart/{slug}/investments-count-by-country"
       params = {}
       
       # Add date filters if provided
       if start_date:
           params['startDate'] = start_date
       if end_date:
           params['endDate'] = end_date
           
       try:
           #logging.info(f"🔍 Fetching investments by country for: {slug}")
           response = requests.get(
               endpoint, 
               headers=self.headers, 
               params=params, 
               timeout=self.timeout
           )
           response.raise_for_status()
           
           json_data = response.json()
           #logging.info("✅ Raw API Response Received")
           
           if not isinstance(json_data, dict) or 'data' not in json_data:
               logging.error("❌ Invalid response structure")
               return None
           
           return self._transform_country_data(json_data['data'])
           
       except Exception as e:
           logging.error(f"❌ Error fetching country investments: {str(e)}")
           return None



        # Add to the FundAPITester class
    
    def _transform_country_data(self, country_data: list) -> dict:
        """Transform country data with maximum safety checks"""
        processed = {
            'countries': [],
            'totals': {
                'normal': 0,
                'lead': 0,
                'all': 0
            }
        }
        
        if not isinstance(country_data, list):
            logging.error("❌ Country data is not a list")
            return processed
        
        for country in country_data:
            try:
                # Safely extract country info
                country_info = {
                    'name': 'Unknown',
                    'code': 'XX',
                    'normal': 0,
                    'lead': 0,
                    'categories': []
                }
                
                # Get country details
                if isinstance(country.get('country'), dict):
                    country_info['name'] = country['country'].get('name', 'Unknown')
                    country_info['code'] = country['country'].get('code', 'XX')
                
                # Get investment counts
                country_info['normal'] = int(country.get('normalCount', 0))
                country_info['lead'] = int(country.get('leadCount', 0))
                
                # Process categories if they exist
                if isinstance(country.get('categories'), list):
                    for category in country['categories']:
                        if isinstance(category, dict):
                            try:
                                percent = float(category.get('percent', 0)) * 100
                                country_info['categories'].append({
                                    'name': str(category.get('name', 'Unknown')),
                                    'count': int(category.get('count', 0)),
                                    'percent': round(percent, 2)
                                })
                            except (ValueError, TypeError):
                                continue
                
                processed['countries'].append(country_info)
                processed['totals']['normal'] += country_info['normal']
                processed['totals']['lead'] += country_info['lead']
                processed['totals']['all'] += country_info['normal'] + country_info['lead']
                
            except Exception as e:
                logging.warning(f"⚠️ Skipped country due to error: {str(e)}")
                continue
        
        # Sort countries by total investments (normal + lead)
        processed['countries'].sort(
            key=lambda x: x['normal'] + x['lead'],
            reverse=True
        )
        
        return processed

    def display_country_results(self, data):
        """Display the processed country results"""
        if not data or not isinstance(data, dict) or not data.get('countries'):
            print("❌ No valid country data to display")
            return
        
        #print(f"\n🌍 Total Investments: {data['totals']['normal']} (Normal) + {data['totals']['lead']} (Lead)")
        
        # Prepare top countries table
        headers = ["Rank", "Country", "Code", "Investments", "Lead", "% of Total"]
        table_data = []
        
        total_all = max(1, data['totals']['all'])  # Avoid division by zero
        
        for idx, country in enumerate(data['countries'][:15], 1):
            country_total = country['normal'] + country['lead']
            percent = (country_total / total_all) * 100
            
            table_data.append([
                idx,
                country['name'],
                country['code'],
                country['normal'],
                country['lead'],
                f"{percent:.1f}%"
            ])
        
        print("\n🏆 Top Countries by Investment Count:")
        print(tabulate(table_data, headers=headers, tablefmt="pretty"))
        
        # Show category breakdown for countries that have categories
        print("\n📊 Category Breakdown:")
        shown = 0
        for country in data['countries']:
            if country.get('categories') and shown < 5:  # Limit to top 5 with categories
                print(f"\n{country['name']} ({country['code']}):")
                cat_headers = ["Category", "Count", "%"]
                cat_table = []
                
                for cat in sorted(country['categories'], key=lambda x: x['percent'], reverse=True):
                    cat_table.append([cat['name'], cat['count'], f"{cat['percent']}%"])
                
                print(tabulate(cat_table, headers=cat_headers, tablefmt="pretty"))
                shown += 1



        # Add to the FundAPITester class
    def get_investments_by_month(self, slug: str, start_date: str, end_date: str):
        """Fetch monthly investment statistics with date filtering"""
        url = f"{self.base_url}/fund-chart/{slug}/investments-count-by-month"
        params = {
            "groupBy": "month",
            "startDate": start_date,
            "endDate": end_date,
            "slug": slug
        }

        try:
            response = requests.get(url, headers=self.headers, params=params, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in investments response")
                return {}

            return self._transform_investment_data(json_data)
        except Exception as e:
            print(f"Investments API Error: {str(e)}")
            return {}

    def _transform_investment_data(self, raw_data: dict) -> dict:
        """Transform investment timeline data"""
        transformed = {
            'months': [],
            'totals': {
                'normal': raw_data.get('normalCount', 0),
                'lead': raw_data.get('leadCount', 0)
            }
        }

        for entry in raw_data.get('data', []):
            # Convert ISO date to "YYYY-MM" format
            date_str = entry.get('date', '')[:7]  # Gets "YYYY-MM" from ISO string

            transformed['months'].append({
                'period': date_str,
                'normal': entry.get('normalCount', 0),
                'lead': entry.get('leadCount', 0),
                'total': entry.get('normalCount', 0) + entry.get('leadCount', 0)
            })

        return transformed



        # Add to the FundAPITester class
    def get_news(self, slug: str, limit=10):
        """Fetch and transform complete news data"""
        url = f"{self.base_url}/news?lang=en&limit={limit}&fundsSlugs={slug}"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()
            return self._transform_news_data(json_data.get('data', []))
        except Exception as e:
            print(f"News API Error: {str(e)}")
            return []

    def _transform_news_data(self, raw_news: list) -> list:
        """Transform all fields from the news API response"""
        transformed = []
        for item in raw_news:
            # Handle potential missing fields with .get()
            news_item = {
                # Core metadata
                'id': item.get('id'),
                'url_id': item.get('urlId'),
                'date': self._format_timestamp(item.get('date')),
                'title': item.get('title'),
                'description': item.get('description'),
                'url': item.get('url'),

                # Media
                'image': item.get('imageUrl'),
                'fallback_image': item.get('fallbackImageUrl'),

                # Content details
                'reading_time': self._format_reading_time(item.get('readingTimeMinutes')),
                'source': {
                    'name': item.get('source', {}).get('name'),
                    'lang': item.get('source', {}).get('lang')
                },

                # Relationships
                'coins': [
                    {
                        'key': coin.get('key'),
                        'symbol': coin.get('symbol'),
                        'name': coin.get('name')
                    } for coin in item.get('relatedCoins', [])
                ],
                'tags': [
                    {
                        'id': tag.get('id'),
                        'key': tag.get('key'),
                        'name': tag.get('name')
                    } for tag in item.get('tags', [])
                ],

                # Flags
                'is_ad': item.get('isAdvertisement', False)
            }
            transformed.append(news_item)
        return transformed

    def _format_timestamp(self, ts: int) -> str:
        """Convert millisecond timestamp to readable date"""
        if not ts: return "Date N/A"
        try:
            return datetime.fromtimestamp(ts/1000).strftime("%b %d, %Y %H:%M")
        except:
            return "Invalid Date"

    def _format_reading_time(self, minutes: [str,float,int]) -> str:
        """Safely format reading time"""
        try:
            return f"{float(minutes):.1f} min"
        except (TypeError, ValueError):
            return "N/A"


        # Add to the FundAPITester class
    def get_co_funds(self, fund_id: int):
        """Fetch and transform co-funding partners data"""
        url = f"{self.base_url}/coin-funds/{fund_id}/co-funds"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in co-funds response")
                return []

            return self._transform_co_funds_data(json_data['data'])
        except Exception as e:
            print(f"Co-Funds API Error: {str(e)}")
            return []

    def _transform_co_funds_data(self, raw_co_funds: list) -> list:
        """Transform co-funding partners data"""
        transformed = []
        for fund in raw_co_funds:
            transformed_fund = {
                'id': fund.get('fundId'),
                'slug': fund.get('slug'),
                'name': fund.get('name'),
                'logo': fund.get('logo'),
                'tier': fund.get('tier'),
                'type': fund.get('fundType'),
                'location': fund.get('location'),
                'stats': {
                    'co_investments': fund.get('coInvestmentsCount', 0),
                    'lead_investments': fund.get('leadCount', 0)
                },
                'investment_types': [
                    {'type': k, 'count': v}
                    for k, v in fund.get('types', {}).items()
                ],
                'categories': [
                    {'category': k, 'count': v}
                    for k, v in fund.get('categories', {}).items()
                ],
                'tags': fund.get('tags', []),
                'crowdsales': fund.get('crowdsalesTypes', []),
                'countries': fund.get('countries', []),
                'raise_ranges': fund.get('raises', [])
            }
            transformed.append(transformed_fund)
        return sorted(transformed, key=lambda x: x['stats']['co_investments'], reverse=True)


        # Add to the FundAPITester class
    def get_avg_round_raise(self, slug: str):
        """Fetch and transform average round raise data properly"""
        endpoint = f"{self.base_url}/coin-funds/avg-round-raise/{slug}"
        
        try:
            #logging.info(f"🔍 Fetching avg round raise for: {slug}")
            response = requests.get(endpoint, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            
            json_data = response.json()
            
            if 'data' not in json_data:
                logging.error("❌ No 'data' field in response")
                return None
            
            return self._process_round_data(json_data['data'])
            
        except Exception as e:
            logging.error(f"❌ API Error fetching avg round raise: {str(e)}")
            return None

    def _process_round_data(self, raw_rounds: list) -> list:
        """Helper: Process and format average round raise data"""
        def format_range(min_val, max_val):
            if max_val is None:
                return f"${min_val:,.0f}+"
            return f"${min_val:,.0f} - ${max_val:,.0f}"
        
        processed = []
        for r in raw_rounds:
            min_val = r.get('raiseFrom', 0)
            max_val = r.get('raiseTo')
            processed.append({
                'range': format_range(min_val, max_val),
                'percentage': float(r.get('percent', 0)),
                'min': min_val,
                'max': max_val
            })
        
        # Sort by minimum value
        return sorted(processed, key=lambda x: x['min'])
    
    
    def test_avg_round_raise(slug: str):
    
        base_url = "https://api.cryptorank.io/v0"
        endpoint = f"{base_url}/coin-funds/avg-round-raise/{slug}"
        
        try:
            print(f"🔍 Testing API endpoint for: {slug}")
            response = requests.get(endpoint, timeout=10)
            response.raise_for_status()
            
            json_data = response.json()
            print("✅ Raw API Response:")
            print(json_data)
            
            if 'data' not in json_data:
                print("❌ No 'data' field in response")
                return None
            
            return process_round_data(json_data['data'])
            
        except requests.exceptions.RequestException as e:
            print(f"❌ API Error: {str(e)}")
            return None

    
  

        # Add to the FundAPITester class
    def get_recent_funding(self, slug: str):
        """Fetch and transform recent funding rounds"""
        url = f"{self.base_url}/coin-funds/recent-funding-rounds/{slug}"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in funding rounds response")
                return []

            return self._transform_funding_data(json_data['data'])
        except Exception as e:
            print(f"Funding Rounds API Error: {str(e)}")
            return []

    def _transform_funding_data(self, raw_funding: list) -> list:
        """Transform and format funding round data"""
        transformed = []
        for round in raw_funding:
            # Parse and format date
            raw_date = round.get('date')
            formatted_date = None
            try:
                date_obj = datetime.fromisoformat(raw_date.replace('Z', '+00:00'))
                formatted_date = date_obj.strftime("%b %d, %Y")
            except:
                formatted_date = "Date N/A"

            transformed.append({
                'date': formatted_date,
                'company': {
                    'name': round.get('coinName'),
                    'key': round.get('coinKey'),
                    'logo': round.get('coinLogo')
                },
                'amount': round.get('raise', 0),
                'stage': round.get('stage', '').title().replace('_', ' '),
                'category': {
                    'name': round.get('categoryName'),
                    'slug': round.get('categorySlug')
                },
                'formatted_amount': f"${round.get('raise', 0):,}",
                'timeline': f"{formatted_date} • {round.get('stage', '').title()}"
            })

        # Sort by date descending
        return sorted(transformed,
                    key=lambda x: x['date'],
                    reverse=True)


    def get_preferred_stages(self, slug: str):
        """Fetch and transform investment stage preferences"""
        url = f"{self.base_url}/coin-funds/preferred-stage/{slug}"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in preferred stage response")
                return []

            return self._transform_stage_data(json_data['data'])
        except Exception as e:
            print(f"Preferred Stage API Error: {str(e)}")
            return []

    def _transform_stage_data(self, raw_stages: list) -> list:
        """Transform and sort stage preference data"""
        transformed = [
            {
                'stage': stage.get('type', 'Unknown').title(),
                'percentage': float(stage.get('percent', 0))
            }
            for stage in raw_stages
        ]
        return sorted(transformed, key=lambda x: x['percentage'], reverse=True)



    # Add to the FundAPITester class
    def get_funding_countries(self, slug: str):
        """Fetch and transform funding countries data"""
        url = f"{self.base_url}/coin-funds/main-funding-countries/{slug}"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in funding countries response")
                return []

            return self._transform_countries_data(json_data['data'])
        except Exception as e:
            print(f"Funding Countries API Error: {str(e)}")
            return []

    def _transform_countries_data(self, raw_countries: list) -> list:
        """Transform country funding data"""
        return sorted([
            {
                'country': country.get('country'),
                'investments': country.get('count', 0)
            }
            for country in raw_countries
        ], key=lambda x: x['investments'], reverse=True)

    # Add to the FundAPITester class
    def get_social_activity(self, fund_id: int):
        """Fetch and transform social activity data"""
        url = f"{self.base_url}/coin-funds/{fund_id}/social-activity"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in social activity response")
                return None

            return self._transform_social_data(json_data['data'])
        except Exception as e:
            print(f"Social Activity API Error: {str(e)}")
            return None

    def _transform_social_data(self, raw_social: dict) -> dict:
        """Transform social activity data"""
        return {
            'profile_url': raw_social.get('link_url'),
            'twitter': {
                'username': raw_social.get('twitter_username'),
                'score': raw_social.get('twitter_score'),
                'followers': raw_social.get('followers_count'),
                'latest_followings': raw_social.get('latest_followings', [])
            }
        }

        # Add to the FundAPITester class
    def get_team_data(self, slug: str):
        """Fetch and transform team data"""
        url = f"{self.base_url}/team/by-fund-key/{slug}"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in team API response")
                return []

            return self._transform_team_data(json_data['data'])
        except Exception as e:
            print(f"Team API Error: {str(e)}")
            return []

    def _transform_team_data(self, raw_team: list) -> list:
        """Transform team member data"""
        return [
            {
                'name': member.get('name'),
                'photo': member.get('logo'),
                'roles': member.get('jobs', []),
                'links': {
                    link['type']: link['value']
                    for link in member.get('links', [])
                    if 'type' in link and 'value' in link
                }
            }
            for member in raw_team
        ]


    def get_fund_data(self, slug: str):
        """Fetch and transform complete fund data"""
        url = f"{self.base_url}/coin-funds/by-slug/{slug}/?locale=en"
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()
            json_data = response.json()

            if 'data' not in json_data:
                print("❌ Missing data in API response")
                return None

            return self._transform_complete_data(json_data['data'])
        except Exception as e:
            print(f"Error: {str(e)}")
            return None

    def _transform_complete_data(self, raw_data: dict) -> dict:
        """Transform all fields from the API response"""
        return {
            # Basic Info
            'basic': {
                'id': raw_data.get('id'),
                'slug': raw_data.get('slug'),
                'name': raw_data.get('name'),
                'tier': raw_data.get('tier'),
                'category': raw_data.get('category', {}),
                'country': raw_data.get('country'),
                'tabs': raw_data.get('tabs', [])
            },

            # Performance Metrics
            'performance': {
                'investments': raw_data.get('investments'),
                'roi': raw_data.get('roi'),
                'categories_distribution': [
                    {
                        'name': cat.get('name'),
                        'count': cat.get('count'),
                        'percentage': cat.get('percentage')
                    } for cat in raw_data.get('categoriesDistribution', [])
                ]
            },

            # Content
            'content': {
                'short_description': raw_data.get('shortDescription'),
                'description': html.unescape(raw_data.get('description', ''))
            },

            # Media Assets
            'media': {
                **raw_data.get('images', {}),
                'logos': {
                    'x60': raw_data.get('images', {}).get('x60'),
                    'x150': raw_data.get('images', {}).get('x150'),
                    'icon': raw_data.get('images', {}).get('icon'),
                    'native': raw_data.get('images', {}).get('native')
                }
            },

            # Investments
            'investments': {
                'top': [
                    {
                        'name': inv.get('name'),
                        'key': inv.get('key'),
                        'logo': inv.get('logo'),
                        'raise': inv.get('raise'),
                        'hidden': inv.get('isHidden')
                    } for inv in raw_data.get('topInvestments', [])
                ]
            },

            # Social Links
            'links': {
                link['type']: link['value']
                for link in raw_data.get('links', [])
                if 'type' in link and 'value' in link
            }
        }


In [11]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from tabulate import tabulate

def test_avg_round_and_country_investments():
    """Test and display average round raise + country investments data"""
    tester = FundAPITester()
    test_slug = "coinbase-ventures"  # Example fund slug

    # 1. Average Round Raise Test
    print("\n=== Testing Average Round Raise ===")
    avg_round_data = tester.get_avg_round_raise(test_slug)

    if avg_round_data:
        print("\n📊 Average Funding Rounds Data:")
        headers = ["Range", "Percentage", "Min ($)", "Max ($)"]
        table_data = [
            [
                item['range'],
                f"{item['percentage']:.2f}%",
                f"{item['min']:,}",
                f"{item['max']:,}" if item['max'] else "N/A"
            ]
            for item in avg_round_data
        ]
        print(tabulate(table_data, headers=headers, tablefmt="pretty"))
    else:
        print("❌ Failed to get average round raise data")

    # 2. Investments By Country Test
    print("\n=== Testing Investments by Country ===")
    country_data = tester.get_investments_by_country(test_slug)

    if country_data:
        print("\n🌍 Country Investment Summary:")
        print(f"Total Normal Investments: {country_data['totals']['normal']:,}")
        print(f"Total Lead Investments: {country_data['totals']['lead']:,}")
        print(f"Total Investments (All): {country_data['totals']['all']:,}")

        headers = ["Rank", "Country", "Code", "Normal", "Lead", "Total", "% of Total"]
        table_data = []
        total_all = max(1, country_data['totals']['all'])  # Avoid division by zero

        for idx, country in enumerate(country_data['countries'][:10], 1):  # Show top 10 countries
            country_total = country['normal'] + country['lead']
            percent = (country_total / total_all) * 100
            table_data.append([
                idx,
                country['name'],
                country['code'],
                country['normal'],
                country['lead'],
                country_total,
                f"{percent:.1f}%"
            ])

        print("\n🏆 Top Countries by Investment Count:")
        print(tabulate(table_data, headers=headers, tablefmt="pretty"))

    else:
        print("❌ Failed to get country investment data")


if __name__ == "__main__":
    test_avg_round_and_country_investments()


INFO:root:🔍 Fetching avg round raise for: coinbase-ventures



=== Testing Average Round Raise ===


INFO:root:🔍 Fetching investments by country for: coinbase-ventures



📊 Average Funding Rounds Data:
+---------------------------+------------+------------+------------+
|           Range           | Percentage |  Min ($)   |  Max ($)   |
+---------------------------+------------+------------+------------+
|      $0 - $1,000,000      |   0.00%    |     0      | 1,000,000  |
|  $1,000,000 - $3,000,000  |   15.00%   | 1,000,000  | 3,000,000  |
| $3,000,000 - $10,000,000  |   40.00%   | 3,000,000  | 10,000,000 |
| $10,000,000 - $20,000,000 |   20.00%   | 10,000,000 | 20,000,000 |
| $20,000,000 - $50,000,000 |   25.00%   | 20,000,000 | 50,000,000 |
|       $50,000,000+        |   0.00%    | 50,000,000 |    N/A     |
+---------------------------+------------+------------+------------+

=== Testing Investments by Country ===


INFO:root:✅ Raw API Response Received



🌍 Country Investment Summary:
Total Normal Investments: 332
Total Lead Investments: 23
Total Investments (All): 355

🏆 Top Countries by Investment Count:
+------+----------------+------+--------+------+-------+------------+
| Rank |    Country     | Code | Normal | Lead | Total | % of Total |
+------+----------------+------+--------+------+-------+------------+
|  1   | United States  |  US  |  202   |  13  |  215  |   60.6%    |
|  2   | United Kingdom |  GB  |   18   |  1   |  19   |    5.4%    |
|  3   |   Singapore    |  SG  |   15   |  1   |  16   |    4.5%    |
|  4   |     India      |  IN  |   12   |  3   |  15   |    4.2%    |
|  5   |    Germany     |  DE  |   9    |  1   |  10   |    2.8%    |
|  6   |     Canada     |  CA  |   10   |  0   |  10   |    2.8%    |
|  7   |   Hong Kong    |  HK  |   9    |  0   |   9   |    2.5%    |
|  8   |  Switzerland   |  CH  |   6    |  1   |   7   |    2.0%    |
|  9   |   Australia    |  AU  |   6    |  0   |   6   |    1.7%    |
|  10

In [44]:
import pandas as pd
import os
import time
import requests
from tqdm import tqdm
from requests.exceptions import RequestException
from math import ceil

def safe_request(func, *args, retries=3, delay=0.5, **kwargs):
    """Retries a function up to 'retries' times with a delay if it fails"""
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except RequestException as e:
            # no print here
            if attempt < retries - 1:
                time.sleep(delay)
    return None

def fetch_all_fund_data_batch(fund_slugs: list, save_dir: str = "funds_data", batch_size: int = 1000):
    tester = FundAPITester()

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    n_batches = ceil(len(fund_slugs) / batch_size)

    for batch_idx in range(n_batches):
        batch_funds = fund_slugs[batch_idx*batch_size : (batch_idx+1)*batch_size]

        # Only display current batch and progress bar
        with tqdm(
            total=len(batch_funds),
            desc=f"Batch {batch_idx+1}/{n_batches}",
            ncols=100,
            bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}"
        ) as pbar:

            batch_country_investments = []
            batch_avg_rounds = []
            batch_recent_funding = []
            batch_news = []
            batch_team = []
            batch_co_funds = []
            batch_fund_info = []

            for slug in batch_funds:
                try:
                    fund_info = safe_request(tester.get_fund_data, slug)
                    if fund_info:
                        basic = fund_info.get('basic', {})
                        basic['fund_slug'] = slug
                        batch_fund_info.append(basic)

                    country_investments = safe_request(tester.get_investments_by_country, slug)
                    if country_investments and 'countries' in country_investments:
                        for country in country_investments['countries']:
                            country['fund_slug'] = slug
                            batch_country_investments.append(country)

                    avg_round = safe_request(tester.get_avg_round_raise, slug)
                    if avg_round:
                        for round_data in avg_round:
                            round_data['fund_slug'] = slug
                            batch_avg_rounds.append(round_data)

                    recent_funding = safe_request(tester.get_recent_funding, slug)
                    if recent_funding:
                        for funding in recent_funding:
                            funding['fund_slug'] = slug
                            batch_recent_funding.append(funding)

                    news_items = safe_request(tester.get_news, slug, limit=10)
                    if news_items:
                        for news in news_items:
                            news['fund_slug'] = slug
                            batch_news.append(news)

                    team_members = safe_request(tester.get_team_data, slug)
                    if team_members:
                        for member in team_members:
                            member['fund_slug'] = slug
                            batch_team.append(member)

                    if fund_info and fund_info.get('basic', {}).get('id'):
                        fund_id = fund_info['basic']['id']
                        co_funds = safe_request(tester.get_co_funds, fund_id)
                        if co_funds:
                            for partner in co_funds:
                                partner['fund_slug'] = slug
                                batch_co_funds.append(partner)

                except Exception:
                    pass  # completely silent on error

                time.sleep(0.5)
                pbar.update(1)

        # Save batch results
        batch_folder = os.path.join(save_dir, f"batch_{batch_idx}")
        os.makedirs(batch_folder, exist_ok=True)

        pd.DataFrame(batch_fund_info).to_csv(os.path.join(batch_folder, "funds_info_2.csv"), index=False)
        pd.DataFrame(batch_country_investments).to_csv(os.path.join(batch_folder, "country_investments_2.csv"), index=False)
        pd.DataFrame(batch_avg_rounds).to_csv(os.path.join(batch_folder, "avg_rounds_2.csv"), index=False)
        pd.DataFrame(batch_recent_funding).to_csv(os.path.join(batch_folder, "recent_funding_2.csv"), index=False)
        pd.DataFrame(batch_news).to_csv(os.path.join(batch_folder, "funds_news_2.csv"), index=False)
        pd.DataFrame(batch_team).to_csv(os.path.join(batch_folder, "funds_team_2.csv"), index=False)
        pd.DataFrame(batch_co_funds).to_csv(os.path.join(batch_folder, "co_funds_2.csv"), index=False)

    print("\n🎯 ALL batches done!")


In [34]:
funds_details_path = "funds_details.csv"  # Update the path if necessary

# Load the CSV file
funds_df = pd.read_csv(funds_details_path)

# Extract the 'slug' column and transform it into a list
fund_slugs = funds_df['slug'].dropna().tolist()

print(fund_slugs)

['hash-key-capital', 'coinbase-ventures', 'delphi-ventures', 'circle', 'a16z-csx', 'polychain-capital', 'sandeep-nailwal', 'spartan-group', 'galaxy-digital', 'stani-kulechov', 'andreessen-horowitz', 'vaneck', 'dragon-fly-capital', 'balaji-srinivasan', 'paradigm', 'sequoia-capital', 'binance-labs', 'santiago-santos', 'bryan-pellegrino', 'consensys', 'multicoin-capital', 'black-rock', 'raj-gokal', 'pantera-capital', 'paul-veradittakit', 'vitalik-buterin', 'blockchain-capital', 'alex-svanevik', 'y-combinator', 'arthur-hayes', 'animoca-brands', 'quantstamp', 'jump-crypto', 'amber-group', 'greenfield-one', 'tether', 'polygon-ventures', 'mh-ventures', 'fenbushi-capital', 'robot-ventures', 'infinity-ventures-crypto', '6th-man-ventures', 'immutable', 'hypersphereventures', 'lightspeed-venture-partners', 'rockaway-blockchain-fund', 'alliance-dao', 'framework-ventures', 'mirana-ventures', 'electric-capital', 'standard-crypto', 'stellardf', 'coin-fund', 'variant', 'unionsquare', 'ribbit-capital',

In [36]:
first_1000_fund_slugs = fund_slugs[:1000]
print(len(first_1000_fund_slugs))  # Should print 1000

1000


In [ ]:
second_1000_fund_slugs = fund_slugs[1000:2000]
second_1000_fund_slugs_df = pd.DataFrame(second_1000_fund_slugs, columns=["slug"])
print(second_1000_fund_slugs_df)


                  slug
0               avaloq
1            kns-group
2             choco-up
3         heat-capital
4                 okex
..                 ...
995             upside
996  ouroboros-capital
997           beraland
998   boba-tea-capital
999      good-partners

[1000 rows x 1 columns]


In [48]:
second_1000_fund_slugs = second_1000_fund_slugs_df['slug'].tolist()

In [51]:
third_1000_fund_slugs = fund_slugs[2000:3000]
third_1000_fund_slugs_df = pd.DataFrame(third_1000_fund_slugs, columns=["slug"])
print(third_1000_fund_slugs_df)


                         slug
0                       3base
1                      block0
2                  xbto-group
3    perpetual-value-partners
4                 foundership
..                        ...
995                    atomic
996             spice-capital
997                      era7
998      hummingbirds-capital
999             mouro-capital

[1000 rows x 1 columns]


In [52]:
third_1000_fund_slugs = third_1000_fund_slugs_df['slug'].tolist()

In [55]:
fourth_1000_fund_slugs = fund_slugs[3000:6000]
fourth_1000_fund_slugs_df = pd.DataFrame(fourth_1000_fund_slugs, columns=["slug"])
print(fourth_1000_fund_slugs_df)
fourth_1000_fund_slugs = fourth_1000_fund_slugs_df['slug'].tolist()

                    slug
0             k5ventures
1                  atrum
2                  psyfi
3                   abra
4          1982-ventures
...                  ...
2995            redstone
2996                 f10
2997  sovereigns-capital
2998        t-rowe-price
2999     bin-zayed-group

[3000 rows x 1 columns]


In [56]:
if __name__ == "__main__":
    fetch_all_fund_data(fourth_1000_fund_slugs, save_dir="funds_data_4")


Fetching funds:   0%|          | 0/3000 [00:00<?, ?it/s]


=== Fetching data for fund: k5ventures ===
Error: argument of type 'NoneType' is not iterable
Team API Error: 404 Client Error: Not Found for url: https://api.cryptorank.io/v0/team/by-fund-key/k5ventures


Fetching funds:   0%|          | 1/3000 [00:02<1:53:52,  2.28s/it]


=== Fetching data for fund: atrum ===
Error: argument of type 'NoneType' is not iterable
Team API Error: 404 Client Error: Not Found for url: https://api.cryptorank.io/v0/team/by-fund-key/atrum


Fetching funds:   0%|          | 2/3000 [00:04<1:50:58,  2.22s/it]

...


=== Fetching data for fund: t-rowe-price ===
Error: argument of type 'NoneType' is not iterable
Team API Error: 404 Client Error: Not Found for url: https://api.cryptorank.io/v0/team/by-fund-key/t-rowe-price


Fetching funds: 100%|█████████▉| 2999/3000 [2:13:24<00:02,  2.22s/it]


=== Fetching data for fund: bin-zayed-group ===
Error: argument of type 'NoneType' is not iterable
Team API Error: 404 Client Error: Not Found for url: https://api.cryptorank.io/v0/team/by-fund-key/bin-zayed-group


Fetching funds: 100%|██████████| 3000/3000 [2:13:27<00:00,  2.67s/it]



✅ All data successfully fetched and saved to: funds_data_4
